In [1]:
import sys
import gzip
import time
import pickle
import numpy as np

import theano
from theano import tensor as T

import lasagne
from lasagne.updates import nesterov_momentum, adam
from lasagne.layers import helper
from models import ResNet_FullPre_Wide as ResNet
from utils import load_pickle_data_cv, batch_iterator_valid, batch_iterator_train_crop_flip
%matplotlib inline
from matplotlib import pyplot

In [2]:
X = T.tensor4('X')
Y = T.ivector('y')

In [3]:
depth = 6
width = 10
output_layer = ResNet(X, n=depth, k=width)

In [4]:
feature_layer = lasagne.layers.get_all_layers(output_layer)[-2]
feature = lasagne.layers.get_output(feature_layer)

In [5]:
lasagne.layers.get_output_shape(feature_layer)

(None, 640)

In [6]:
train_X, test_X, train_y, test_y = load_pickle_data_cv()

[[[ 130.68655396  130.10546875  131.00730896 ...,  131.13044739
    130.46455383  130.2585144 ]
  [ 130.07684326  129.30218506  130.16917419 ...,  130.11120605
    129.40142822  129.24234009]
  [ 129.69766235  128.66070557  129.42695618 ...,  129.22239685
    128.60726929  128.64715576]
  ..., 
  [ 126.5687561   124.60520172  123.93291473 ...,  124.20082092
    124.64533234  125.49713135]
  [ 127.11519623  125.53757477  125.33624268 ...,  125.46095276
    125.54613495  125.95500183]
  [ 127.80984497  126.69049072  126.95684814 ...,  126.73840332
    126.52495575  126.59886932]]

 [[ 136.05722046  135.43437195  136.2149353  ...,  136.3591156
    135.76197815  135.53715515]
  [ 135.30397034  134.44544983  135.14733887 ...,  135.1741333
    134.60409546  134.44535828]
  [ 134.64828491  133.46908569  134.04020691 ...,  133.9420929
    133.53291321  133.59544373]
  ..., 
  [ 125.98253632  123.77459717  122.83339691 ...,  122.98991394
    123.72742462  124.86979675]
  [ 126.43002319  124.664

In [7]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [8]:
get_features = theano.function(inputs=[X], outputs=feature)

In [9]:
X_feature = np.zeros((45000,640))
i = 0
batch_size = 500
for batch in iterate_minibatches(train_X, train_y, batch_size, shuffle=False):
    inputs, targets = batch
    b = get_features(inputs)
    X_feature[batch_size*i:batch_size*(i+1)] = b
    i+=1
    print(i)
np.savez('./train_data.npz', X_feature)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90


In [11]:
test_feature = np.zeros((5000,640))
i = 0
batch_size = 500
for batch in iterate_minibatches(test_X, test_y, batch_size, shuffle=False):
    inputs, targets = batch
    b = get_features(inputs)
    X_feature[batch_size*i:batch_size*(i+1)] = b
    i+=1
    print(i)
np.savez('./test_data.npz', test_feature)

1
2
3
4
5
6
7
8
9
10


In [17]:
def train_and_eval( model, train_x, train_y, test_x, test_y ):
    model.fit( train_x, train_y )
    p = model.predict( test_x )
    OA = sum(test_y==p)/len(test_y)
    return OA
import sklearn
Svm=svm.SVC(kernel='linear',C=1,shrinking=False)
svm_auc = train_and_eval( Svm, X_feature, train_y, \
        test_feature, test_y )


In [18]:
svm_auc

0.105